In [12]:
bucket = 'sagemaker-multi-label-test'
prefix = 'ic-multilabel'

s3train = 's3://{}/{}/train/'.format(bucket, prefix)
s3validation = 's3://{}/{}/validation/'.format(bucket, prefix)

In [13]:
import sagemaker
from sagemaker import get_execution_role
from sagemaker.amazon.amazon_estimator import image_uris

role = get_execution_role()
sess = sagemaker.Session()

print(sess.boto_session.client('sts').get_caller_identity()['Arn'])
print(role)


arn:aws:sts::466363126778:assumed-role/AWSGlueServiceSageMakerNotebookRole/SageMaker
arn:aws:iam::466363126778:role/AWSGlueServiceSageMakerNotebookRole


In [14]:
train_data = sagemaker.inputs.TrainingInput(
    s3train, 
    distribution='FullyReplicated', 
    content_type='application/x-recordio', 
    s3_data_type='S3Prefix'
)

validation_data = sagemaker.inputs.TrainingInput(
    s3validation, 
    distribution='FullyReplicated', 
    content_type='application/x-recordio', 
    s3_data_type='S3Prefix'
)

data_channels = {'train': train_data, 'validation': validation_data}

In [15]:
s3_output_location = 's3://{}/{}/output'.format(bucket, prefix)

In [16]:
from sagemaker.image_uris import retrieve, config_for_framework
training_image = retrieve('image-classification', sess.boto_region_name, )
print (training_image)

685385470294.dkr.ecr.eu-west-1.amazonaws.com/image-classification:1


In [17]:
multilabel_ic = sagemaker.estimator.Estimator(
                                             training_image,
                                             role, 
                                             instance_count=1, 
                                             instance_type='ml.p3.2xlarge',
                                             output_path=s3_output_location,
                                             sagemaker_session=sess
                                             )

In [18]:
multilabel_ic.set_hyperparameters(
                                 num_layers=18,
                                 image_shape = "3,224,224",
                                 num_classes=10,
                                 resize=256,
                                 epochs=100,
                                 top_k='2',
                                 num_training_samples=3457,
                                 use_weighted_loss=1,
                                 augmentation_type = 'crop_color_transform',
                                 precision_dtype='float32',
                                 multi_label=1,
                                 optimizer='adam',
                                 learning_rate=0.000021,
                                 mini_batch_size=16,
                                )

In [19]:
multilabel_ic.fit(inputs=data_channels, logs=True)

2021-03-02 02:58:45 Starting - Starting the training job...
2021-03-02 02:59:09 Starting - Launching requested ML instancesProfilerReport-1614653925: InProgress
......
2021-03-02 03:00:11 Starting - Preparing the instances for training......
2021-03-02 03:01:12 Downloading - Downloading input data...
2021-03-02 03:01:30 Training - Downloading the training image.....Docker entrypoint called with argument(s): train
[03/02/2021 03:02:26 INFO 140047524992128] Reading default configuration from /opt/amazon/lib/python2.7/site-packages/image_classification/default-input.json: {u'beta_1': 0.9, u'gamma': 0.9, u'beta_2': 0.999, u'optimizer': u'sgd', u'use_pretrained_model': 0, u'eps': 1e-08, u'epochs': 30, u'lr_scheduler_factor': 0.1, u'num_layers': 152, u'image_shape': u'3,224,224', u'precision_dtype': u'float32', u'mini_batch_size': 32, u'weight_decay': 0.0001, u'learning_rate': 0.1, u'momentum': 0}
[03/02/2021 03:02:26 INFO 140047524992128] Merging with provided configuration from /opt/ml/inp

[03/02/2021 03:02:52 INFO 140047524992128] Epoch[2] Batch [120]#011Speed: 551.839 samples/sec#011accuracy=0.539618
[03/02/2021 03:02:53 INFO 140047524992128] Epoch[2] Batch [140]#011Speed: 557.134 samples/sec#011accuracy=0.539495
[03/02/2021 03:02:53 INFO 140047524992128] Epoch[2] Batch [160]#011Speed: 554.747 samples/sec#011accuracy=0.540062
[03/02/2021 03:02:54 INFO 140047524992128] Epoch[2] Batch [180]#011Speed: 544.178 samples/sec#011accuracy=0.540608
[03/02/2021 03:02:54 INFO 140047524992128] Epoch[2] Batch [200]#011Speed: 549.241 samples/sec#011accuracy=0.542475
[03/02/2021 03:02:55 INFO 140047524992128] Epoch[2] Train-accuracy=0.543866
[03/02/2021 03:02:55 INFO 140047524992128] Epoch[2] Time cost=6.302
[03/02/2021 03:02:56 INFO 140047524992128] Epoch[2] Validation-accuracy=0.582527
[03/02/2021 03:02:56 INFO 140047524992128] Storing the best model with validation accuracy: 0.582527
[03/02/2021 03:02:56 INFO 140047524992128] Saved checkpoint to "/opt/ml/model/image-classification-

[03/02/2021 03:03:32 INFO 140047524992128] Epoch[7] Batch [160]#011Speed: 547.365 samples/sec#011accuracy=0.608191
[03/02/2021 03:03:33 INFO 140047524992128] Epoch[7] Batch [180]#011Speed: 545.327 samples/sec#011accuracy=0.607251
[03/02/2021 03:03:33 INFO 140047524992128] Epoch[7] Batch [200]#011Speed: 550.289 samples/sec#011accuracy=0.608520
[03/02/2021 03:03:34 INFO 140047524992128] Epoch[7] Train-accuracy=0.609635
[03/02/2021 03:03:34 INFO 140047524992128] Epoch[7] Time cost=6.321
[03/02/2021 03:03:35 INFO 140047524992128] Epoch[7] Validation-accuracy=0.628696
[03/02/2021 03:03:35 INFO 140047524992128] Storing the best model with validation accuracy: 0.628696
[03/02/2021 03:03:35 INFO 140047524992128] Saved checkpoint to "/opt/ml/model/image-classification-0008.params"
[03/02/2021 03:03:36 INFO 140047524992128] Epoch[8] Batch [20]#011Speed: 600.644 samples/sec#011accuracy=0.614583
[03/02/2021 03:03:37 INFO 140047524992128] Epoch[8] Batch [40]#011Speed: 547.226 samples/sec#011accurac

[03/02/2021 03:04:14 INFO 140047524992128] Epoch[12] Validation-accuracy=0.656182
[03/02/2021 03:04:14 INFO 140047524992128] Storing the best model with validation accuracy: 0.656182
[03/02/2021 03:04:14 INFO 140047524992128] Saved checkpoint to "/opt/ml/model/image-classification-0013.params"
[03/02/2021 03:04:15 INFO 140047524992128] Epoch[13] Batch [20]#011Speed: 593.443 samples/sec#011accuracy=0.636905
[03/02/2021 03:04:15 INFO 140047524992128] Epoch[13] Batch [40]#011Speed: 537.076 samples/sec#011accuracy=0.640701
[03/02/2021 03:04:16 INFO 140047524992128] Epoch[13] Batch [60]#011Speed: 535.934 samples/sec#011accuracy=0.641086
[03/02/2021 03:04:17 INFO 140047524992128] Epoch[13] Batch [80]#011Speed: 551.708 samples/sec#011accuracy=0.641049
[03/02/2021 03:04:17 INFO 140047524992128] Epoch[13] Batch [100]#011Speed: 543.193 samples/sec#011accuracy=0.639418
[03/02/2021 03:04:18 INFO 140047524992128] Epoch[13] Batch [120]#011Speed: 538.919 samples/sec#011accuracy=0.638791
[03/02/2021 0

[03/02/2021 03:04:53 INFO 140047524992128] Saved checkpoint to "/opt/ml/model/image-classification-0018.params"
[03/02/2021 03:04:54 INFO 140047524992128] Epoch[18] Batch [20]#011Speed: 597.997 samples/sec#011accuracy=0.658333
[03/02/2021 03:04:55 INFO 140047524992128] Epoch[18] Batch [40]#011Speed: 564.832 samples/sec#011accuracy=0.651677
[03/02/2021 03:04:55 INFO 140047524992128] Epoch[18] Batch [60]#011Speed: 556.214 samples/sec#011accuracy=0.652971
[03/02/2021 03:04:56 INFO 140047524992128] Epoch[18] Batch [80]#011Speed: 565.889 samples/sec#011accuracy=0.652623
[03/02/2021 03:04:56 INFO 140047524992128] Epoch[18] Batch [100]#011Speed: 554.757 samples/sec#011accuracy=0.652661
[03/02/2021 03:04:57 INFO 140047524992128] Epoch[18] Batch [120]#011Speed: 542.908 samples/sec#011accuracy=0.651860
[03/02/2021 03:04:57 INFO 140047524992128] Epoch[18] Batch [140]#011Speed: 550.000 samples/sec#011accuracy=0.652571
[03/02/2021 03:04:58 INFO 140047524992128] Epoch[18] Batch [160]#011Speed: 545.9

...

[03/02/2021 03:11:40 INFO 140047524992128] Epoch[70] Batch [120]#011Speed: 530.603 samples/sec#011accuracy=0.760847
[03/02/2021 03:11:41 INFO 140047524992128] Epoch[70] Batch [140]#011Speed: 539.714 samples/sec#011accuracy=0.759619
[03/02/2021 03:11:41 INFO 140047524992128] Epoch[70] Batch [160]#011Speed: 535.650 samples/sec#011accuracy=0.759550
[03/02/2021 03:11:42 INFO 140047524992128] Epoch[70] Batch [180]#011Speed: 531.115 samples/sec#011accuracy=0.761153
[03/02/2021 03:11:42 INFO 140047524992128] Epoch[70] Batch [200]#011Speed: 537.165 samples/sec#011accuracy=0.762127
[03/02/2021 03:11:43 INFO 140047524992128] Epoch[70] Train-accuracy=0.762876
[03/02/2021 03:11:43 INFO 140047524992128] Epoch[70] Time cost=6.451
[03/02/2021 03:11:44 INFO 140047524992128] Epoch[70] Validation-accuracy=0.746603
[03/02/2021 03:11:45 INFO 140047524992128] Epoch[71] Batch [20]#011Speed: 595.847 samples/sec#011accuracy=0.772024
[03/02/2021 03:11:45 INFO 140047524992128] Epoch[71] Batch [40]#011Speed: 562

[03/02/2021 03:12:30 INFO 140047524992128] Epoch[76] Validation-accuracy=0.750815
[03/02/2021 03:12:31 INFO 140047524992128] Epoch[77] Batch [20]#011Speed: 594.495 samples/sec#011accuracy=0.766964
[03/02/2021 03:12:31 INFO 140047524992128] Epoch[77] Batch [40]#011Speed: 552.138 samples/sec#011accuracy=0.767378
[03/02/2021 03:12:32 INFO 140047524992128] Epoch[77] Batch [60]#011Speed: 539.770 samples/sec#011accuracy=0.768033
[03/02/2021 03:12:33 INFO 140047524992128] Epoch[77] Batch [80]#011Speed: 555.546 samples/sec#011accuracy=0.766821
[03/02/2021 03:12:33 INFO 140047524992128] Epoch[77] Batch [100]#011Speed: 549.068 samples/sec#011accuracy=0.768007
[03/02/2021 03:12:34 INFO 140047524992128] Epoch[77] Batch [120]#011Speed: 544.893 samples/sec#011accuracy=0.766064
[03/02/2021 03:12:34 INFO 140047524992128] Epoch[77] Batch [140]#011Speed: 551.406 samples/sec#011accuracy=0.764982
[03/02/2021 03:12:35 INFO 140047524992128] Epoch[77] Batch [160]#011Speed: 546.841 samples/sec#011accuracy=0.7

[03/02/2021 03:13:20 INFO 140047524992128] Epoch[83] Batch [100]#011Speed: 550.663 samples/sec#011accuracy=0.774567
[03/02/2021 03:13:21 INFO 140047524992128] Epoch[83] Batch [120]#011Speed: 539.978 samples/sec#011accuracy=0.773037
[03/02/2021 03:13:21 INFO 140047524992128] Epoch[83] Batch [140]#011Speed: 548.899 samples/sec#011accuracy=0.773803
[03/02/2021 03:13:22 INFO 140047524992128] Epoch[83] Batch [160]#011Speed: 546.391 samples/sec#011accuracy=0.773486
[03/02/2021 03:13:22 INFO 140047524992128] Epoch[83] Batch [180]#011Speed: 538.763 samples/sec#011accuracy=0.774448
[03/02/2021 03:13:23 INFO 140047524992128] Epoch[83] Batch [200]#011Speed: 544.467 samples/sec#011accuracy=0.776399
[03/02/2021 03:13:23 INFO 140047524992128] Epoch[83] Train-accuracy=0.776852
[03/02/2021 03:13:23 INFO 140047524992128] Epoch[83] Time cost=6.361
[03/02/2021 03:13:24 INFO 140047524992128] Epoch[83] Validation-accuracy=0.750136
[03/02/2021 03:13:25 INFO 140047524992128] Epoch[84] Batch [20]#011Speed: 59

[03/02/2021 03:14:10 INFO 140047524992128] Epoch[89] Train-accuracy=0.785880
[03/02/2021 03:14:10 INFO 140047524992128] Epoch[89] Time cost=6.433
[03/02/2021 03:14:11 INFO 140047524992128] Epoch[89] Validation-accuracy=0.762024
[03/02/2021 03:14:11 INFO 140047524992128] Epoch[90] Batch [20]#011Speed: 591.363 samples/sec#011accuracy=0.791369
[03/02/2021 03:14:12 INFO 140047524992128] Epoch[90] Batch [40]#011Speed: 560.852 samples/sec#011accuracy=0.786585
[03/02/2021 03:14:13 INFO 140047524992128] Epoch[90] Batch [60]#011Speed: 552.079 samples/sec#011accuracy=0.787807
[03/02/2021 03:14:13 INFO 140047524992128] Epoch[90] Batch [80]#011Speed: 563.739 samples/sec#011accuracy=0.790046
[03/02/2021 03:14:14 INFO 140047524992128] Epoch[90] Batch [100]#011Speed: 556.968 samples/sec#011accuracy=0.788428
[03/02/2021 03:14:14 INFO 140047524992128] Epoch[90] Batch [120]#011Speed: 553.219 samples/sec#011accuracy=0.788946
[03/02/2021 03:14:15 INFO 140047524992128] Epoch[90] Batch [140]#011Speed: 559.3

[03/02/2021 03:14:50 INFO 140047524992128] Epoch[95] Batch [20]#011Speed: 590.106 samples/sec#011accuracy=0.792262
[03/02/2021 03:14:51 INFO 140047524992128] Epoch[95] Batch [40]#011Speed: 549.460 samples/sec#011accuracy=0.787195
[03/02/2021 03:14:51 INFO 140047524992128] Epoch[95] Batch [60]#011Speed: 533.441 samples/sec#011accuracy=0.796209
[03/02/2021 03:14:52 INFO 140047524992128] Epoch[95] Batch [80]#011Speed: 547.728 samples/sec#011accuracy=0.796914
[03/02/2021 03:14:52 INFO 140047524992128] Epoch[95] Batch [100]#011Speed: 542.972 samples/sec#011accuracy=0.796720
[03/02/2021 03:14:53 INFO 140047524992128] Epoch[95] Batch [120]#011Speed: 537.305 samples/sec#011accuracy=0.796643
[03/02/2021 03:14:53 INFO 140047524992128] Epoch[95] Batch [140]#011Speed: 547.323 samples/sec#011accuracy=0.794902
[03/02/2021 03:14:54 INFO 140047524992128] Epoch[95] Batch [160]#011Speed: 537.260 samples/sec#011accuracy=0.795458
[03/02/2021 03:14:55 INFO 140047524992128] Epoch[95] Batch [180]#011Speed: 5

In [20]:
ic_classifier = multilabel_ic.deploy(initial_instance_count = 1,
                                          instance_type = 'ml.m4.xlarge')

---------------!

In [24]:
import json
import boto3
 
endpoint = 'image-classification-2021-03-02-03-31-36-334' 
runtime = boto3.Session().client('sagemaker-runtime')
 
image = 'flower1.png'
# Read image into memory
with open(image, 'rb') as f:
    payload = f.read()
# Send image via InvokeEndpoint API
response = runtime.invoke_endpoint(EndpointName=endpoint, ContentType='application/x-image', Body=payload)

# Unpack response
result = json.loads(response['Body'].read().decode())

In [25]:
print(result)

[0.9886089563369751, 0.6623129844665527, 0.003996745683252811, 0.235959991812706, 0.10212567448616028, 0.3281160891056061, 0.02550269290804863, 0.10979624837636948, 0.6117209196090698, 0.23091921210289001]


In [26]:
ic_classifier.delete_endpoint()